First we have to resample our 5 years data depending on the moving averages.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
the_path = os.getcwd()
data_path = os.path.join(the_path, '../data/processed-data/')
file_name = 'processed.csv'
file_path = data_path + file_name
df = pd.read_csv(file_path, index_col='Date')
df.drop(df.columns[df.columns.str.contains('unnamed: 0',case = False)],axis = 1,inplace = True)
df.shape

(3425987, 1)

After testing we have defined that 4000 samples is the optimal amount of sample for safe handling.
That means, depending on the frequency of the sampling is the lenght of the data:
 - Freq:  1 min ->   2.78 days
 - Freq:  5 min ->  13.89 days
 - Freq: 10 min ->  27.78 days
 - Freq: 15 min ->  41.67 days
 - Freq: 30 min ->  83.33 days
 - Freq: 60 min -> 166.67 days
 - Freq:  2 hrs -> 333.33 days
 - Freq:  3 hrs ->  1.36  years
 - Freq:  5 hrs ->   2    years
 - Freq:  8 hrs ->   2    years
 - Freq: 12 hrs ->   2    years
 - Freq: 24 hrs ->   2    years

In [3]:
L_df = []
L_freq = ['5min', '10min', '15min', '30min', '60min', '2H', '3H', '5H', '8H', '12H', '24H']

# 5 min
range_5min = pd.date_range(end=df.index[-1], periods=4000, freq='5T')
df_5min = df[str(range_5min[0]):]
L_df.append(df_5min)
# 10 min
range_10min = pd.date_range(end=df.index[-1], periods=4000, freq='10T')
df_10min = df[str(range_10min[0]):]
L_df.append(df_10min)
# 15 min
range_15min = pd.date_range(end=df.index[-1], periods=4000, freq='15T')
df_15min = df[str(range_15min[0]):]
L_df.append(df_15min)
# 30 min
range_30min = pd.date_range(end=df.index[-1], periods=4000, freq='30T')
df_30min = df[str(range_30min[0]):]
L_df.append(df_30min)
# 60 min
range_60min = pd.date_range(end=df.index[-1], periods=4000, freq='60T')
df_60min = df[str(range_60min[0]):]
L_df.append(df_60min)
# 2 hrs
range_2hrs = pd.date_range(end=df.index[-1], periods=4000, freq='2H')
df_2hrs = df[str(range_2hrs[0]):]
L_df.append(df_2hrs)
# 3 hrs
range_3hrs = pd.date_range(end=df.index[-1], periods=4000, freq='3H')
df_3hrs = df[str(range_3hrs[0]):]
L_df.append(df_3hrs)
# 5 hrs
df_5hrs = df[str(pd.Timestamp(df.index[-1]) - pd.offsets.Day(730)):]
L_df.append(df_5hrs)
# 8 hrs
df_8hrs = df[str(pd.Timestamp(df.index[-1]) - pd.offsets.Day(730)):]
L_df.append(df_8hrs)
# 12 hrs
df_12hrs = df[str(pd.Timestamp(df.index[-1]) - pd.offsets.Day(730)):]
L_df.append(df_12hrs)
# 24 hrs
df_24hrs = df[str(pd.Timestamp(df.index[-1]) - pd.offsets.Day(730)):]
L_df.append(df_24hrs)

In [6]:
for dataframe, frequency in zip(L_df, L_freq):
    values = dataframe['Average'].values
    ts = pd.Series(values, index=dataframe.index)
    ts.index = pd.to_datetime(ts.index)
    ts = ts[~ts.index.duplicated()]
    ts[ts==-1] = np.nan
    newFreq=ts.resample(frequency).asfreq()
    new = (ts.reindex(ts.index | newFreq.index)
           .interpolate(method='time')
           .loc[newFreq.index])
    r_df = pd.DataFrame({'Date':new.index, 'Average':new.values})
    print(r_df.head(5))
    file_name = 'resampled_' + frequency + '.csv'
    file_path = data_path + file_name
    r_df.to_csv(file_path)
print('DONE!')

    Average                Date
0  7137.425 2018-08-06 02:45:00
1  7132.340 2018-08-06 02:50:00
2  7132.510 2018-08-06 02:55:00
3  7129.990 2018-08-06 03:00:00
4  7134.555 2018-08-06 03:05:00
    Average                Date
0  7660.080 2018-07-23 05:30:00
1  7652.585 2018-07-23 05:40:00
2  7643.110 2018-07-23 05:50:00
3  7649.715 2018-07-23 06:00:00
4  7664.475 2018-07-23 06:10:00
    Average                Date
0  6729.285 2018-07-09 08:15:00
1  6725.840 2018-07-09 08:30:00
2  6722.855 2018-07-09 08:45:00
3  6719.985 2018-07-09 09:00:00
4  6714.050 2018-07-09 09:15:00
    Average                Date
0  7238.115 2018-05-28 16:30:00
1  7221.355 2018-05-28 17:00:00
2  7222.230 2018-05-28 17:30:00
3  7240.890 2018-05-28 18:00:00
4  7239.615 2018-05-28 18:30:00
     Average                Date
0  11253.290 2018-03-06 09:00:00
1  11261.450 2018-03-06 10:00:00
2  10949.030 2018-03-06 11:00:00
3  10932.540 2018-03-06 12:00:00
4  10909.495 2018-03-06 13:00:00
    Average                Date
0 